In [ ]:
import numpy as np
import pandas as pd
import numbers

# Point #1

In [ ]:
import numpy as np

def PCA(D, red):
    
    data_centralizada = D - np.mean(D, axis=0)
    
    # Matriz covarianza
    cov_matrix = np.cov(data_centralizada, rowvar=False)
    
    # Eigenvalores de la matriz
    eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
    
    # Ordenar los más importantes
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors = eigenvectors[:, sorted_indices]


    #Determinar el número de dimensiones a retener en función de la fracción de varianza
    total_variance = np.sum(eigenvalues)
    if red < 1:
        
        retained_variance = red * total_variance
        cum_variance = np.cumsum(eigenvalues)
        num_dimensions = np.argmax(cum_variance >= retained_variance) + 1
    elif isinstance(red, int) and red >= 1:
        # Usar los que de red
        num_dimensions = red
    else:
        raise ValueError("Valor inválido, recuerde que debe ser entero positivo.")    
    
    # Usar los mayores de 'num_dimensions' eigenvectors
    selected_eigenvectors = eigenvectors[:, :num_dimensions]
    
    # Datos reducidos (producto punto)
    reduced_data = np.dot(data_centralizada, selected_eigenvectors)
    
    # Varianza explicada en los datos reducidos
    #Varianza Explicada= Suma de todos los valores propios/Suma de los primeros k valores propios
    varianza_explicada = np.sum(eigenvalues[:num_dimensions]) / total_variance
    
    return reduced_data, varianza_explicada



In [ ]:
df_iris = pd.read_csv("iris.csv")
df_baseball_numeric = pd.read_csv("baseball_numeric.csv")
df_faber = pd.read_csv("fabert.csv")
dfAmazon = pd.read_csv("amazon.csv")

# Point #2

## Item A


In [ ]:
#Apply PCA

## Item B


In [ ]:
#Now do PCA for a reduction to 1, 2 and 3 dimensions

# Point #3


In [ ]:
df_cats = pd.read_csv("noisy_cats.csv")

## Item A

In [ ]:
#Replace the categorical attribute with Bernoulli

## Item B


In [ ]:
#Scatter plot for the points of the classes

## Item C

In [ ]:
#Reduce the data to 1 dimensions, then scatter again.

## Item D

In [ ]:
#Transform the data, reduce to 1 dimension again, then scatter

## Item E

In [ ]:
#Centralize the columns, divide by the standard deviation, apply PCA.